In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
config_file = '../config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [9]:
m_ab_file = '../'+expand(config['lr']['talon']['ab'], species='mouse')[0]
m_filt_ab_file = '../'+expand(config['lr']['talon']['filt_ab'], species='mouse')[0]

In [10]:
df2 = pd.read_csv(m_filt_ab_file, sep='\t')
df[['annot_transcript_id', 'gene_novelty']].groupby('gene_novelty').count()

FileNotFoundError: [Errno 2] No such file or directory: '../data/mouse/lr/talon/filt_talon_abundance_filtered.tsv'

In [4]:
df = pd.read_csv(m_ab_file, sep='\t')

In [6]:
df.gene_novelty.unique()

array(['Known', 'Antisense', 'Intergenic', 'Fusion'], dtype=object)

In [7]:
df[['annot_gene_id', 'gene_novelty']].groupby('gene_novelty').count().reset_index()

,gene_novelty,annot_gene_id
0,Antisense,222873
1,Fusion,43465
2,Intergenic,64034
3,Known,2552024


In [8]:
df.loc[df.gene_novelty=='Fusion']

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length,gene_novelty,transcript_novelty,...,adrenal_gland_1_2,cortex_5x_f_1_2,cortex_14d_f_2,cortex_2mo_m_2,gastroc_14d_f_2,gastroc_25d_m_2,gastroc_4d_f_2,heart_18-20mo_m_1,hippocampus_wt_m_2_1,hippocampus_18-20mo_f_2
64257,55774,142467,TALONG000055774,TALONT000142467,TALONG000055774,TALONT000142467,13,1934,Fusion,Fusion,...,0,0,0,0,0,0,0,0,0,0
64383,55808,142593,TALONG000055808,TALONT000142593,TALONG000055808,TALONT000142593,9,3331,Fusion,Fusion,...,0,1,1,0,0,0,0,0,2,1
64754,55869,142964,TALONG000055869,TALONT000142964,TALONG000055869,TALONT000142964,8,2352,Fusion,Fusion,...,0,2,0,0,0,0,0,0,1,0
64757,55869,142967,TALONG000055869,TALONT000142967,TALONG000055869,TALONT000142967,9,2518,Fusion,NIC,...,0,3,0,0,0,0,0,0,0,0
64759,55869,142969,TALONG000055869,TALONT000142969,TALONG000055869,TALONT000142969,4,1035,Fusion,NIC,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882153,127991,2960363,TALONG000127991,TALONT002960363,TALONG000127991,TALONT002960363,1,1207,Fusion,Genomic,...,1,0,0,0,1,0,0,0,0,0
2882155,127991,2960365,TALONG000127991,TALONT002960365,TALONG000127991,TALONT002960365,6,1507,Fusion,NNC,...,0,0,0,0,0,0,0,0,1,0
2882291,180891,2960501,TALONG000180891,TALONT002960501,TALONG000180891,TALONT002960501,18,3855,Fusion,NIC,...,0,0,0,1,0,0,0,0,0,0
2882323,180891,2960533,TALONG000180891,TALONT002960533,TALONG000180891,TALONT002960533,1,1894,Fusion,Genomic,...,1,0,0,0,0,0,0,0,0,0
